In [ ]:
# pip install numpy matplotlib jupyter pandas opencv-python scikit-image ipywidgets ipympl
import os
import cv2
import numpy as np
import pandas as pd
from skimage import measure, morphology
from skimage.io import imread
from tkinter import filedialog, Tk
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
from scipy.ndimage import convolve
plt.rcParams['figure.figsize'] = [8, 8]

GUI_INPUT = False

# Set path and min-max paramters
if GUI_INPUT:
    Tk().withdraw()
    input_folder = filedialog.askdirectory(title="Choose input folder")
    output_file = filedialog.asksaveasfilename(defaultextension=".csv", title="Choose where to save results", initialfile="Intensity_Results.csv")
    min_val = float(input("Enter intensity minimum (e.g. 0): "))
    max_val = float(input("Enter intensity maximum (e.g. 16000): "))
else:
    input_folder = os.getcwd() # Current folder
    output_file = os.path.join(input_folder, "Intensity_Results.csv")
    min_val = 0
    max_val = 16000

# Collect image files
files = sorted(os.listdir(input_folder))
filtered_files = [f for f in files if f.endswith("tiff")]

# Save image files to list
images = []
for j in range(0, len(filtered_files), 2):
    int_path = os.path.join(input_folder, filtered_files[j])
    nuc_path = os.path.join(input_folder, filtered_files[j+1])
    int_img = imread(int_path, as_gray=True)
    nuc_img = imread(nuc_path, as_gray=True)
    name = filtered_files[j].split(".")[0]

    # Scale intensity image and Gaussian blur
    int_img = (np.clip(int_img, min_val, max_val) - min_val) / (max_val - min_val)
    background = cv2.GaussianBlur(int_img, (91, 91), 0)
    int_img = cv2.subtract(int_img, background)
    images.append([name, int_img, nuc_img, None, None])
    

In [ ]:
GUI_IMAGES = False

def get_default_params(name):
    match name[0]:
        case "A":
            return 800, 500
        case "B":
            return 800, 500
        case "C":
            return 500, 500
        case "D":
            return 500, 500
        case "E":
            return 400, 300
        case "F":
            return 400, 300
        case "G":
            return 400, 300
        case "H":
            return 400, 300
        case _:
            return min(500, max_val/10), 300

def get_binary(nuc_img, min_thresh, maxthresh, min_size, mineccentricity, minsolidity):
    binary = np.logical_and(nuc_img > min_thresh, nuc_img < maxthresh).astype(np.uint8)
    binary = morphology.remove_small_objects(binary.astype(bool), min_size=min_size)
    binary = morphology.remove_small_holes(binary, area_threshold=min_size)

    label_img = measure.label(binary)
    props = measure.regionprops(label_img)
    for prop in props:
        if prop.eccentricity < mineccentricity or prop.solidity < minsolidity:
            binary[label_img == prop.label] = False

    return binary

%matplotlib inline
def GUI_binary(entry):
    name = entry[0]
    nuc_img = entry[2]
    slider_minthresh_initial, slider_minsize_initial = get_default_params(name)

    slider_minthresh = widgets.IntSlider(value=slider_minthresh_initial , min=min_val, max=max_val/10, step=10, description="Min Threshold:", continuous_update=False)
    slider_maxthresh = widgets.IntSlider(value=max_val, min=max_val/10, max=max_val, step=10, description="Max Threshold:", continuous_update=False)
    slider_minsize = widgets.IntSlider(value=slider_minsize_initial, min=0, max=800, step=10, description="Min size:", continuous_update=False)
    slider_mineccentricity = widgets.FloatSlider(value=0.6, min=0, max=1, step=0.01, description="Min Eccentricity:", continuous_update=False)
    slider_minsolidity = widgets.FloatSlider(value=0.9, min=0, max=1, step=0.01, description="Min Solidity:", continuous_update=False)
    button = widgets.Button(description="Confirm", button_style="success")
    output = widgets.Output()

    plt.close()
    plt.figure()

    def update_image(change=None):
        with output:
            binary = get_binary(nuc_img, slider_minthresh.value, slider_maxthresh.value, slider_minsize.value, slider_mineccentricity.value, slider_minsolidity.value)
            clear_output(wait=True)
            plt.title(name)
            plt.axis('off')
            plt.subplots_adjust(left=0, right=1, top=1, bottom=0)
            plt.imshow(binary , cmap='gray')
            plt.show()

    def on_confirm_clicked(b=None):
        """Store the confirmed value and clear the plot"""
        binary = get_binary(slider_minthresh.value, slider_maxthresh.value, slider_minsize.value, slider_mineccentricity.value, slider_minsolidity.value)
        plt.close()
        container.close()
        display(f"Confirmed values for \"{name}\": min_thresh = {slider_minthresh.value}, max_thresh = {slider_maxthresh.value}, min_size = {slider_minsize.value}, mineccentricity = {slider_mineccentricity.value}, minsolidity = {slider_minsolidity.value}")
        entry[3] = binary
        entry[4] = {"min_thresh": slider_minthresh.value, "max_thresh": slider_maxthresh.value, "min_size": slider_minsize.value, "mineccentricity": slider_mineccentricity.value, "minsolidity": slider_minsolidity.value}

    slider_minthresh.observe(update_image, names="value")
    slider_maxthresh.observe(update_image, names="value")
    slider_minsize.observe(update_image, names="value")
    slider_mineccentricity.observe(update_image, names="value")
    slider_minsolidity.observe(update_image, names="value")
    container = widgets.VBox([slider_minthresh, slider_maxthresh, slider_minsize, slider_mineccentricity, slider_minsolidity, button, output])
    display(container)
    update_image()
    button.on_click(on_confirm_clicked)

if GUI_IMAGES:
    for entry in images:
        GUI_binary(entry)
else:
    for entry in images:
        name = entry[0]
        nuc_img = entry[2]
        slider_minthresh_initial, slider_minsize_initial = get_default_params(name)
        mineccentricity = 0.6
        minsolidity = 0.9

        binary = get_binary(nuc_img, slider_minthresh_initial, max_val, slider_minsize_initial, mineccentricity, minsolidity)
        print(f"Confirmed values for \"{name}\": min_thresh = {slider_minthresh_initial}, max_thresh = {max_val}, min_size = {slider_minsize_initial}, mineccentricity = {mineccentricity}, minsolidity = {minsolidity}")
        entry[3] = binary
        entry[4] = {"min_thresh": slider_minthresh_initial, "max_thresh": max_val, "min_size": slider_minsize_initial, "mineccentricity": mineccentricity, "minsolidity": minsolidity}

In [ ]:
expand_size = 2
shrink_size = 2
intensity_categories = [-0.1, 0.0, 0.09, 0.15, 1.0]

results = []
all_mask_list = []
all_perinuclear_list = []
all_perinuclear_active_list = []
all_perinuclear_inactive_list = []
for entry in images:
    name, int_img, nuc_img, binary, params = entry
    all_mask = False
    all_perinuclear = False
    all_perinuclear_active = False
    all_perinuclear_inactive = False
    active = 0
    inactive = 0

    # Label nuclei
    labels = measure.label(binary)
    props = measure.regionprops(labels)

    for idx, prop in enumerate(props, start=1):
        mask = (labels == prop.label).astype(np.uint8)
        all_mask = np.logical_or(all_mask, mask)

        # Perinuclear region = expand - shrink
        shrink = cv2.erode(mask, np.ones((int(shrink_size*2+1), int(shrink_size*2+1)), np.uint8), iterations=1)
        expand = cv2.dilate(mask, np.ones((int(expand_size*2+1), int(expand_size*2+1)), np.uint8), iterations=1)
        perinuclear = ((expand - shrink) > 0).astype(bool)

        # Get values and measure intensity inside perinuclear mask
        values = int_img[perinuclear]
        mean_intensity = values.mean()
        all_perinuclear = np.logical_or(all_perinuclear, perinuclear) 
        
        number_pixels_in_category = []
        for i in range(len(intensity_categories)):
            number_pixels_in_category.append(int(np.sum(values <= intensity_categories[i])))

        proportion_pixels_in_category = []
        for i in range(len(number_pixels_in_category)):
            if i == 0:
                proportion_pixels_in_category.append(number_pixels_in_category[0])
            else:
                proportion_pixels_in_category.append(number_pixels_in_category[i] - number_pixels_in_category[i-1])
        proportion_pixels_in_category = np.array(proportion_pixels_in_category) / len(values)

        is_active = False
        if proportion_pixels_in_category[3] + proportion_pixels_in_category[4] >= 0.3:
            is_active = True
        elif proportion_pixels_in_category[0] + proportion_pixels_in_category[1] >= 0.2:
            is_active = False
        elif proportion_pixels_in_category[2] + proportion_pixels_in_category[3] + proportion_pixels_in_category[4] >= 0.6:
            is_active = True
        elif proportion_pixels_in_category[0] + proportion_pixels_in_category[1] >= 0.6:
            is_active = False
        else:
            is_active = True
        
        if is_active:
            # Cell is active
            active += 1
            all_perinuclear_active = np.logical_or(all_perinuclear_active, perinuclear)
        else:
            # Cell is inactive
            inactive += 1
            all_perinuclear_inactive = np.logical_or(all_perinuclear_inactive, perinuclear)

    results.append({
        "Name": name,
        "NumCells": active + inactive,
        "NumActive": active,
        "NumInactive": inactive,
        "FractionActive": active / (active + inactive) if active + inactive > 0 else "NaN",
        "FractionInactive": inactive / (active + inactive) if active + inactive > 0 else "NaN",
        "Parameters": str(params)
    })
    
    all_mask_list.append(all_mask)
    all_perinuclear_list.append(all_perinuclear)
    all_perinuclear_active_list.append(all_perinuclear_active)
    all_perinuclear_inactive_list.append(all_perinuclear_inactive)
    print(f"\"{name}\": Active: {active}, inactive: {inactive}. Percentage active: {100.0 * active / (active + inactive) :.1f}%.")

pd.DataFrame(results).to_csv(output_file, index=False, mode="w")
print(f"Analysis complete! Results saved to {output_file}.")


In [ ]:
%matplotlib widget
if False:
    plt.close()
    for i in range(len(images)):
        int_img = images[i][1]
        plt.figure()
        plt.axis('off')
        plt.title(images[i][0])
        plt.imshow(int_img, cmap='gray')
        contours = measure.find_contours(all_mask_list[i], level=0.5)
        for contour in contours:
            plt.plot(contour[:, 1], contour[:, 0], 'r-', linewidth=0.5)
        plt.show()

In [ ]:
%matplotlib widget
from scipy.ndimage import gaussian_filter
plt.close()
for i in range(len(images)):
    int_img = images[i][1]
    plt.figure()
    plt.axis('off')
    plt.title(images[i][0])
    plt.imshow(int_img, cmap='gray')

    if (np.sum(all_perinuclear_active_list[0])) > 0:
        contours_active = measure.find_contours(all_perinuclear_active_list[i], level=0.5)
        for contour in contours_active:
            plt.plot(contour[:, 1], contour[:, 0], 'b-', linewidth=0.5)

    if (np.sum(all_perinuclear_inactive_list[0])) > 0:
        contours_inactive = measure.find_contours(all_perinuclear_inactive_list[i], level=0.5)
        for contour in contours_inactive:
            plt.plot(contour[:, 1], contour[:, 0], 'r-', linewidth=0.5)
    plt.colorbar()
    plt.show()